In [ ]:
# Import necessary libraries
import os
import pickle
from flask import Flask, request, render_template_string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import emoji
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nltk

# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Preprocess reviews function
def preprocess_review(review):
    review = emoji.demojize(review)  # Convert emojis to text
    review = review.lower()  # Lowercase
    tokens = word_tokenize(review)  # Tokenize
    stop_words = set(stopwords.words('english'))  # Stopwords
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    stemmer = PorterStemmer()  # Stemming
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

# Load dataset and preprocess
import pandas as pd
data = pd.read_csv('D:\\projects\\Reviews.csv')  # Replace with your dataset path
data['Review'] = data['Review'].apply(preprocess_review)

# Split data into train and test sets
from sklearn.model_selection import train_test_split
X = data['Review']
y = data['Liked']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train Naive Bayes Model
model = MultinomialNB()
model.fit(X_train_vectorized, y_train)

# Flask app with enhanced styling
app = Flask(__name__)

HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Sentiment Analysis</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: url('https://www.transparenttextures.com/patterns/diamond-upholstery.png'), linear-gradient(135deg, #74b9ff, #0984e3);
            color: #ffffff;
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            background-size: cover;
        }
        .container {
            background: rgba(255, 255, 255, 0.9);
            border-radius: 12px;
            box-shadow: 0 6px 12px rgba(0, 0, 0, 0.2);
            padding: 30px;
            width: 500px;
            text-align: center;
        }
        h1 {
            color: #0984e3;
            margin-bottom: 20px;
            font-size: 28px;
        }
        textarea {
            width: 100%;
            padding: 15px;
            margin-bottom: 20px;
            border: 1px solid #ccc;
            border-radius: 6px;
            resize: none;
            font-size: 16px;
        }
        button {
            background: linear-gradient(135deg, #6c5ce7, #341f97);
            color: white;
            border: none;
            padding: 12px 25px;
            font-size: 16px;
            font-weight: bold;
            border-radius: 8px;
            cursor: pointer;
            transition: all 0.3s ease;
        }
        button:hover {
            background: linear-gradient(135deg, #341f97, #6c5ce7);
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
        }
        .result {
            margin-top: 20px;
            font-size: 20px;
            font-weight: bold;
            color: #2d3436;
            padding: 10px;
            border-radius: 6px;
            background: #dfe6e9;
        }
        footer {
            margin-top: 30px;
            font-size: 14px;
            color: #b2bec3;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Sentiment Analysis</h1>
        <form action="/predict" method="post">
            <textarea id="review" name="review" rows="5" placeholder="Write your review here..."></textarea><br>
            <button type="submit">Analyze Sentiment</button>
        </form>
        {% if prediction %}
        <div class="result">
            {{ prediction }}
        </div>
        {% endif %}
        <footer>Powered by 4298 & 4286 <footer>
    </div>
</body>
</html>
"""

@app.route('/')
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route('/predict', methods=['POST'])
def predict():
    review = request.form['review']
    cleaned_review = preprocess_review(review)
    vectorized_review = vectorizer.transform([cleaned_review])
    prediction = model.predict(vectorized_review)[0]
    sentiment = "Positive 😊" if prediction == 1 else "Negative 😔"
    return render_template_string(HTML_TEMPLATE, prediction=f"The sentiment is: {sentiment}")

# Run Flask app inside Jupyter Notebook and open in browser
import threading
import webbrowser

def open_browser():
    webbrowser.open("http://127.0.0.1:5000", new=1)

def run_flask():
    threading.Timer(2, open_browser).start()  # Open browser after 2 seconds
    app.run(debug=False, use_reloader=False, port=5000)

run_flask()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manog\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\manog\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [11/Jan/2025 11:16:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2025 11:16:04] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Jan/2025 11:16:29] "POST /predict HTTP/1.1" 200 -
